In [1]:
!pip install selenium --quiet
!apt-get update --quiet
!apt install chromium-chromedriver --quiet

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 981 kB 27.3 MB/s 
     |████████████████████████████████| 139 kB 53.3 MB/s 
     |████████████████████████████████| 358 kB 57.4 MB/s 
     |████████████████████████████████| 4.1 MB 46.7 MB/s 
     |████████████████████████████████| 55 kB 3.8 MB/s 
     |████████████████████████████████| 58 kB 5.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.10 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cud

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [2]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
grab_url='https://food.grab.com/sg/en/'


In [5]:
driver.get(grab_url)
location=driver.find_element_by_id("location-input").send_keys("City Square Mall,180 Kitchener Rd, Singapore 208539")
button = driver.find_element_by_name("ant-btn")
button.click()
count = 0
while count < 10:
  try:

    #time.sleep(10)
    button = driver.find_element("ant-btn")
    button.click()
    count = count + 1

  except:
    print("Number of pages scraped: ",count)
    count = 1000000
elements = driver.find_elements_by_xpath("//div[@class ='ant-layout']" and "//div[@class = 'ant-row-flex RestaurantListRow___1SbZY']" and "//div[@class ='ant-col-24 RestaurantListCol___1FZ8V  ant-col-md-12 ant-col-lg-6']" and "//h6[@class = 'name___2epcT']")

AttributeError: ignored

In [ ]:
elements

In [ ]:
res_name = []
for element in elements:
    res_name.append(element.text)

In [ ]:
dataset = pd.DataFrame(res_name, columns = ['Restaurant'])

In [ ]:
url_elem = driver.find_elements_by_xpath("//a[contains(@href, '/sg/en/restaurant')]")
url_list = []
for e in url_elem:
    url_list.append(e.get_attribute("href"))
new_db=pd.DataFrame(url_list, columns = ['url'])
dataset=pd.concat([dataset,url_list],axis=1,join='inner')
dataset['latitude']=''
dataset['longitude']=''

In [ ]:
i = 0

for index, row in dataset.iterrows():
    
    dataset_url= row[1]
    res_page = requests.get(dataset_url)
    
    res_page = BeautifulSoup(res_page.text, 'html.parser')
    test_text = res_page.text.find('latlng":{')
    restaurant_latitude = res_page.text[test_text+20:test_text+30]
    restaurant_longitude =res_page.text[test_text+43:test_text+54]

    print(row, restaurant_latitude, restaurant_longitude)
    dataset.at[i,'latitude'] = restaurant_latitude
    dataset.at[i,'longitude'] = restaurant_longitude

    time.sleep(2)

    i = i + 1

    if i == 1000:
      break